# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 9 2023 1:07PM,254573,10,9010605,"Citieffe, Inc.",Released
1,Jan 9 2023 12:55PM,254572,10,MSP217347,"Methapharm, Inc.",Released
2,Jan 9 2023 12:55PM,254572,10,MSP217349,"Methapharm, Inc.",Released
3,Jan 9 2023 12:55PM,254572,10,MSP217350,"Methapharm, Inc.",Released
4,Jan 9 2023 12:55PM,254572,10,MSP217351,"Methapharm, Inc.",Released
5,Jan 9 2023 12:55PM,254572,10,MSP217352,"Methapharm, Inc.",Released
6,Jan 9 2023 12:55PM,254572,10,MSP217353,"Methapharm, Inc.",Released
7,Jan 9 2023 12:32PM,254571,10,Yusen-9010599,Yusen – 3D Matrix,Released
8,Jan 9 2023 12:20PM,254569,10,0086286247,ISDIN Corporation,Released
9,Jan 9 2023 12:20PM,254569,10,0086286248,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,254568,Released,24
28,254569,Released,21
29,254571,Released,1
30,254572,Released,6
31,254573,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254568,NaN,NaN,24.0
254569,NaN,NaN,21.0
254571,NaN,NaN,1.0
254572,NaN,NaN,6.0
254573,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254505,8.0,1.0,2.0
254511,0.0,4.0,4.0
254512,0.0,0.0,11.0
254516,2.0,1.0,6.0
254533,2.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254505,8,1,2
254511,0,4,4
254512,0,0,11
254516,2,1,6
254533,2,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254505,8,1,2
1,254511,0,4,4
2,254512,0,0,11
3,254516,2,1,6
4,254533,2,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254505,8,1,2
1,254511,,4,4
2,254512,,,11
3,254516,2,1,6
4,254533,2,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 9 2023 1:07PM,254573,10,"Citieffe, Inc."
1,Jan 9 2023 12:55PM,254572,10,"Methapharm, Inc."
7,Jan 9 2023 12:32PM,254571,10,Yusen – 3D Matrix
8,Jan 9 2023 12:20PM,254569,10,ISDIN Corporation
29,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC"
53,Jan 9 2023 11:57AM,254567,19,VITABIOTICS USA INC.
63,Jan 9 2023 11:54AM,254566,15,"Brookfield Pharmaceuticals, LLC"
99,Jan 9 2023 11:37AM,254565,10,"Methapharm, Inc."
102,Jan 9 2023 11:16AM,254563,15,"Alliance Pharma, Inc."
142,Jan 9 2023 10:55AM,254562,20,"HVL, LLC dba Atrium Innovations"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 9 2023 1:07PM,254573,10,"Citieffe, Inc.",,,1
1,Jan 9 2023 12:55PM,254572,10,"Methapharm, Inc.",,,6
2,Jan 9 2023 12:32PM,254571,10,Yusen – 3D Matrix,,,1
3,Jan 9 2023 12:20PM,254569,10,ISDIN Corporation,,,21
4,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC",,,24
5,Jan 9 2023 11:57AM,254567,19,VITABIOTICS USA INC.,,,10
6,Jan 9 2023 11:54AM,254566,15,"Brookfield Pharmaceuticals, LLC",,,36
7,Jan 9 2023 11:37AM,254565,10,"Methapharm, Inc.",,,3
8,Jan 9 2023 11:16AM,254563,15,"Alliance Pharma, Inc.",,6,34
9,Jan 9 2023 10:55AM,254562,20,"HVL, LLC dba Atrium Innovations",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 1:07PM,254573,10,"Citieffe, Inc.",1,,
1,Jan 9 2023 12:55PM,254572,10,"Methapharm, Inc.",6,,
2,Jan 9 2023 12:32PM,254571,10,Yusen – 3D Matrix,1,,
3,Jan 9 2023 12:20PM,254569,10,ISDIN Corporation,21,,
4,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC",24,,
5,Jan 9 2023 11:57AM,254567,19,VITABIOTICS USA INC.,10,,
6,Jan 9 2023 11:54AM,254566,15,"Brookfield Pharmaceuticals, LLC",36,,
7,Jan 9 2023 11:37AM,254565,10,"Methapharm, Inc.",3,,
8,Jan 9 2023 11:16AM,254563,15,"Alliance Pharma, Inc.",34,6,
9,Jan 9 2023 10:55AM,254562,20,"HVL, LLC dba Atrium Innovations",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 1:07PM,254573,10,"Citieffe, Inc.",1,,
1,Jan 9 2023 12:55PM,254572,10,"Methapharm, Inc.",6,,
2,Jan 9 2023 12:32PM,254571,10,Yusen – 3D Matrix,1,,
3,Jan 9 2023 12:20PM,254569,10,ISDIN Corporation,21,,
4,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC",24,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 1:07PM,254573,10,"Citieffe, Inc.",1,NaN,NaN
1,Jan 9 2023 12:55PM,254572,10,"Methapharm, Inc.",6,NaN,NaN
2,Jan 9 2023 12:32PM,254571,10,Yusen – 3D Matrix,1,NaN,NaN
3,Jan 9 2023 12:20PM,254569,10,ISDIN Corporation,21,NaN,NaN
4,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC",24,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 1:07PM,254573,10,"Citieffe, Inc.",1,0.0,0.0
1,Jan 9 2023 12:55PM,254572,10,"Methapharm, Inc.",6,0.0,0.0
2,Jan 9 2023 12:32PM,254571,10,Yusen – 3D Matrix,1,0.0,0.0
3,Jan 9 2023 12:20PM,254569,10,ISDIN Corporation,21,0.0,0.0
4,Jan 9 2023 12:03PM,254568,19,"Innogenix, LLC",24,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4072813,149,3.0,12.0
15,509129,70,6.0,0.0
19,1018158,49,4.0,0.0
20,509120,2,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4072813,149,3.0,12.0
1,15,509129,70,6.0,0.0
2,19,1018158,49,4.0,0.0
3,20,509120,2,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,149,3.0,12.0
1,15,70,6.0,0.0
2,19,49,4.0,0.0
3,20,2,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,149.0
1,15,Released,70.0
2,19,Released,49.0
3,20,Released,2.0
4,10,Executing,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,12.0,0.0,0.0,0.0
Executing,3.0,6.0,4.0,0.0
Released,149.0,70.0,49.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,12.0,0.0,0.0,0.0
1,Executing,3.0,6.0,4.0,0.0
2,Released,149.0,70.0,49.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,12.0,0.0,0.0,0.0
1,Executing,3.0,6.0,4.0,0.0
2,Released,149.0,70.0,49.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()